# `ipywidgets` Interactive Demo

Simple demonstration of rendering Interactive widgets in a `jupyterlite` notebook.

`ipywidgets` can be installed in this deployment (it provides the @jupyter-widgets/jupyterlab-manager federated extension), but you will need to make your own deployment to have access to other interactive widgets libraries.

In [ ]:
import micropip

await micropip.install('ipywidgets')

In [ ]:
from ipywidgets import IntSlider

In [ ]:
slider = IntSlider()

In [ ]:
slider

In [ ]:
slider

In [ ]:
slider.value

In [ ]:
slider.value = 5

In [ ]:
from ipywidgets import IntText, link

In [ ]:
text = IntText()

In [ ]:
text

In [ ]:
link((slider, 'value'), (text, 'value'));

# `bqplot` Interactive Demo

Plotting in JupyterLite

`bqplot` can be installed in this deployment (it provides the bqplot federated extension), but you will need to make your own deployment to have access to other interactive widgets libraries.

In [ ]:
import micropip

await micropip.install('bqplot')

In [ ]:
from bqplot import *

import numpy as np
import pandas as pd

np.random.seed(0)

n = 100

x = list(range(n))
y = np.cumsum(np.random.randn(n)) + 100.

sc_x = LinearScale()
sc_y = LinearScale()

lines = Lines(
    x=x, y=y,
    scales={'x': sc_x, 'y': sc_y}
)
ax_x = Axis(scale=sc_x, label='Index')
ax_y = Axis(scale=sc_y, orientation='vertical', label='lines')

Figure(marks=[lines], axes=[ax_x, ax_y], title='Lines')

In [ ]:
lines.colors = ['green']

In [ ]:
lines.fill = 'bottom'

In [ ]:
lines.marker = 'circle'

In [ ]:
n = 100

x = list(range(n))
y = np.cumsum(np.random.randn(n))

sc_x = LinearScale()
sc_y = LinearScale()

bars = Bars(
    x=x, y=y,
    scales={'x': sc_x, 'y': sc_y}
)
ax_x = Axis(scale=sc_x, label='Index')
ax_y = Axis(scale=sc_y, orientation='vertical', label='bars')

Figure(marks=[bars], axes=[ax_x, ax_y], title='Bars', animation_duration=1000)

In [ ]:
bars.y = np.cumsum(np.random.randn(n))